In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.analysis as analysisTools
import timeit
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Main parameters
numparticles = 3
boxsize = 6 #2.5 #10 #8 #6 #5 
D = 1.0
Drot = 1.0
lagtime = 40 # 100
rootDirectory = '../../data/trimer/first_passage_times/'

In [ ]:
# Color definitions
c1 = '#ff3300' #'xkcd:bright orange' # 'darkorange' #'C1' # '#ff3300' # for benchmark
c2 = '#0099ff' #'royalblue' #'royalblue' # 'C0' # '#0099ff' # for MSM/RD
c1alt = 'xkcd:bright orange'
c2alt = 'royalblue'
# Alpha transparency definition
a1 = 0.2 #0.2
a2 = 0.7 #0.9 #0.7

## Load FPT for trimer formation data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 6000
filename1 = rootDirectory + 'trimerFPTs_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fpts = []
with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fpts.append(time)

# Normalize weights
weights = np.ones_like(fpts)/float(len(fpts))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 6000
filename = rootDirectory + 'MSMRDtrimerFPTs_trajs' + str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
#filename = rootDirectory + 'trimerFPTs_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
MSMRDfpts = []
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfpts.append(time)
            
# Normalize weights
weightsMSMRD = np.ones_like(MSMRDfpts)/float(len(MSMRDfpts))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
calculateRates = False
numBootsrapSamples = 2000
mfpt, fptstd = analysisTools.bootstrapping(fpts, numBootsrapSamples)
print("Raw MFPTs: ", np.array(fpts).mean())
print("Bootstrapping mean and std (A): ", mfpt, fptstd)
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
MSMRDmfpt, MSMRDfptstd = analysisTools.bootstrapping(MSMRDfpts, numBootsrapSamples)
print("Raw MSMRD MFPTs: ",np.array(MSMRDfpts).mean())
print("MSMRD bootstrapping mean and std: ", MSMRDmfpt, MSMRDfptstd)
print("Trajectory lengths")
if calculateRates:
    mkonA = 1.0/mfptA
    mkonB = 1.0/mfptB
    MSMRDmkonA = 1.0/MSMRDmfptA
    MSMRDmkonB = 1.0/MSMRDmfptB
    konAstd = fptAstd/(mfptA*mfptA)
    konBstd = fptBstd/(mfptB*mfptB)
    MSMRDkonAstd = MSMRDfptAstd/(MSMRDmfptA*MSMRDmfptA)
    MSMRDkonBstd = MSMRDfptBstd/(MSMRDmfptB*MSMRDmfptB)
    errorKonA = np.abs(mkonA - MSMRDmkonA)
    relErrorKonA = errorKonA/mkonA
    errorKonB = np.abs(mkonB - MSMRDmkonB)
    relErrorKonB = errorKonB/mkonB
    print("On rates:")
    print("Bootstrapping mean and std (A):", mkonA, konAstd)
    print("Bootstrapping mean and std (B):", mkonB, konBstd)
    print("Bootstrapping MSMRD mean and std (A):", MSMRDmkonA, MSMRDkonAstd)
    print("Bootstrapping MSMRD mean and std (B):", MSMRDmkonB, MSMRDkonBstd)
    print("Error and relative error (A):", errorKonA, relErrorKonA)
    print("Error and relative error (B):", errorKonB, relErrorKonB)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, ax1 = plt.subplots(nrows=1, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])
#hbins = range(0, 500,15)
#hbins = range(0, 400,12)
#hbins = range(0, 200,6)
hbins = range(0, 100,3)
#hbins = range(0, 33,1)


# Plot A to unbound histogram
ax1.hist(fpts, bins = hbins, alpha=a1, color=c1, weights = weights );
ax1.hist(fpts, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weights, label =r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfpts, bins = hbins, alpha=a1, color=c2, weights = weightsMSMRD);
ax1.hist(MSMRDfpts, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$\mathrm{Unbound} \rightarrow A$', title_fontsize = fsize, fancybox=True)
ax1.text(40, 0.075, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfpt, fptstd), fontsize=fsize, color=c1)
ax1.text(50, 0.065, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfpt, MSMRDfptstd), fontsize=fsize, color=c2alt)
ax1.set_yticks(np.arange(0, 0.18, step=0.1));
#ax1.set_xlim([0,33])
#ax1.set_ylim([0,0.18])
ax1.set_xlim([0,100])

plt.savefig('trimerFPTs.pdf', bbox_inches='tight')
plt.savefig('trimerFPTs.svg', bbox_inches='tight')

## Load all unbound to bound FPT data for $K_{on}$ vs concentration plot

In [ ]:
# Load all data
numTrajs = 5000
boxsizes = [6, 7, 8, 9, 10, 11, 12, 13]
numBoxes = len(boxsizes)
fpt = [[] for i in range(numBoxes)]
MSMRDfpt = [[] for i in range(numBoxes)]
for i, boxsize in enumerate(boxsizes):
    filename1 = rootDirectory + '/patchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
    filename2 = rootDirectory + 'MSMRDpatchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
    with open(filename1) as file:
        for line in file:
            state, time = line.split(" ")
            fpt[i].append(float(time))
    with open(filename2) as file:
        for line in file:
            state, time = line.split(" ")
            MSMRDfpt[i].append(float(time))

### Calculation of FPTs and comparisons plots

In [ ]:
# Calculate meand and bootstrapping error
numBootsrapSamples = 1000
numValues = 250
meanKon = np.zeros(numBoxes)
meanKonMSMRD = np.zeros(numBoxes)
stdDevKon = np.zeros(numBoxes)
stdDevKonMSMRD = np.zeros(numBoxes)
errorKon = np.zeros(numBoxes)
errorKonMSMRD = np.zeros(numBoxes)
for i, boxsize in enumerate(boxsizes):
    # Calculate bootstrapping samples of mean first passage times
    mfpt, fptStdDev = analysisTools.bootstrapping(fpt[i], numBootsrapSamples, numValues)
    MSMRDmfpt, MSMRDfptStdDev = analysisTools.bootstrapping(MSMRDfpt[i], numBootsrapSamples, numValues)
    # Calculate on rate corresponding to bootstrapping samples
    meanKon[i], stdDevKon[i] = 1.0/mfpt, fptStdDev/(mfpt*mfpt)
    meanKonMSMRD[i], stdDevKonMSMRD[i] = 1.0/MSMRDmfpt, MSMRDfptStdDev/(MSMRDmfpt*MSMRDmfpt)
    # Percentual errors
    errorKon[i] = stdDevKon[i] #1.96*stdDevKon[i]/numBootsrapSamples
    errorKonMSMRD[i] = stdDevKonMSMRD[i] #1.96*stdDevKonMSMRD[i]/numBootsrapSamples

In [ ]:
# Plot on rates vs concentration:
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 1.0/pow(np.array(boxsizes),3)
# Assuming moleculas have a diameter of 5nm, the actual boxsize is 5*boxisze nm,
# lets convert to Molar concentration
avogadro = 6.022E23
oneLiter2nm = 1.0E24
concentrationsMolar = concentrations*oneLiter2nm/avogadro

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, meanKon, marker='o', markersize=6, linewidth=linewidth , color=c1alt, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor=c1alt, 
                facecolor=c1alt)

# Traditional error bar plots
ax.errorbar(concentrationsMolar, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2alt, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
#ax.set_xlabel(r'$\mathrm{Concentration}$ ($1/\mathrm{Volume}$)', fontsize = fsize+4)
ax.set_xlabel(r'$\mathrm{Concentration}$ ($M$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{on}$)', fontsize = fsize+6)
ax.set_yticks([0.001,.01])
#ax.set_ylim([0.01,0.12])
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
#plt.savefig('kon_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot equilibrium constant vs concentration
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2

# Calculate Keq and errors
Keq = meanKon/meanKoff
KeqMSMRD = meanKonMSMRD/meanKoffMSMRD
errorKeq = Keq*(errorKon/meanKon + koffstd/meanKoff)
errorKeqMSMRD = KeqMSMRD*(errorKonMSMRD/meanKonMSMRD + koffstdMSMRD/meanKoffMSMRD)

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, Keq, marker='o', markersize=6, linewidth=linewidth , color=c1alt, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, Keq-errorKeq, Keq+errorKeq, alpha=0.3, edgecolor=c1alt, facecolor=c1alt)

# Traditional error bar plots
ax.errorbar(concentrationsMolar, KeqMSMRD, yerr=errorKeqMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2alt, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
ax.set_xlabel(r'$\mathrm{Concentration}$ ($M$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{eq}$)', fontsize = fsize+6)
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
#plt.savefig('keq_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot on rates vs concentration, zoomed
fig = plt.figure()
fsize = 16
ax = fig.add_subplot(111)
linewidth = 2

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, meanKon, marker='o', markersize=6, linewidth=linewidth , color='C1', 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor='C1', facecolor='C1')
#ax.plot(concentrations, meanKonMSMRD, marker='o', markersize=6, linewidth=0*linewidth, color='C0', 
#        label=r'$\mathrm{MSM/RD}$)
#ax.fill_between(concentrations, meanKonMSMRD-errorKonMSMRD, meanKonMSMRD+errorKonMSMRD, alpha=0.35, 
#                edgecolor='C0', facecolor='C0')

# Traditional error bar plots
#ax.errorbar(concentrations, meanKon, yerr=errorKon, marker='o', markersize=6, elinewidth=2, 
#            linewidth=linewidth , capsize=5, color='C1', label=r'$\mathrm{Benchmark}$')
ax.errorbar(concentrationsMolar, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color='C0', label=r'$\mathrm{MSM/RD}$')
ax.set_yscale('log')
ax.set_xlim([0.0018,0.0047])
ax.set_ylim([0.003,0.018])
ax.set_xticks([], [])
#plt.savefig('kon_dimer_zoom.pdf', bbox_inches='tight')